<a href="https://colab.research.google.com/github/ovapichi/biomecanica-datos/blob/celda_de_cargas/Celda_por_equipo_Armado_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [32]:

  #CELDA DE CARGAS WIN LABORAT
  #INGRESAR CARPETA DE ARCHIVOS .dat
direccion_crudos="/content/drive/Shareddrives/Sociedad Hebraica Argentina/Noviembre 2023/Datos/Celda/Crudos/"
direccion_final="/content/drive/Shareddrives/Sociedad Hebraica Argentina/Noviembre 2023/Datos/Celda/"
fecha="Noviembre 2023"
df_final = pd.DataFrame()
df_individual = pd.DataFrame()
#Entrega un .csv con formato de columnas ["jugadora","maximo","musculo","tiempo"]

  def leer_carpeta(direccion_crudos):
    lista_archivos=[]
    for nombre_archivo in os.listdir(direccion_crudos):
      lista_archivos.append(nombre_archivo)
    lista_archivos.sort()
    #print(lista_archivos)
    seleccion_muslo(lista_archivos,direccion_crudos)
    return

  def seleccion_muslo(lista,direccion_crudos):
    jugadora=[]
    maximos=[]
    musculo=[]
    i=0
    for archivo in lista:

      df=pd.read_csv(direccion_crudos+"/"+archivo,on_bad_lines='skip',sep = "\t", decimal=",",header=None, encoding="utf-16")
      df=df.drop([0],axis=0)
      df=df.drop([0,1,2], axis=1)
      maximos.append(df[3].max())
#falta el numero del individual
      if archivo[-12:-4]=="Cuad Der":
        musculo.append("Cuadriceps Derechos")
        jugadora.append(archivo[:-13])
        df_individual['cuad_der']=df[3].max()
      elif archivo[-12:-4]=="Cuad Izq":
        musculo.append("Cuadriceps Izquierdos")
        jugadora.append(archivo[:-13])
        df_individual['cuad_izq']=df[3].max()
      elif archivo[-12:-4]==" Isq Der":
        jugadora.append(archivo[:-12])
        musculo.append("Isquios Derechos")
        df_individual['isq_der']=df[3].max()
      elif archivo[-12:-4]==" Isq Izq":
        jugadora.append(archivo[:-12])
        musculo.append("Isquios Izquierdo")
        df_individual['isq_izq']=df[3].max()

      i+=1
      if i%4 ==0:
        df_individual.to_csv(direccion_final+archivo[:-12]+".csv")

    df_final['jugadora']=jugadora
    df_final['maximos']=maximos
    df_final['musculo']=musculo
    df_final['tiempo']=5
    return




leer_carpeta(direccion_crudos)
df_final.to_csv(direccion_final+"Celda"+fecha+".csv")
df_final

,jugadora,maximos,musculo,tiempo
0,Agustina Fridman,64.364,Cuadriceps Derechos,5
1,Agustina Fridman,62.957,Cuadriceps Izquierdos,5
2,Agustina Fridman,32.568,Isquios Derechos,5
3,Agustina Fridman,32.069,Isquios Izquierdo,5
4,Aixa Martin,65.442,Cuadriceps Derechos,5
...,...,...,...,...
101,Sofia Bez,25.830,Isquios Izquierdo,5
102,Sofia Hercmann,51.765,Cuadriceps Derechos,5
103,Sofia Hercmann,50.418,Cuadriceps Izquierdos,5
104,Sofia Hercmann,32.170,Isquios Derechos,5


In [31]:

#función para la aplicación de fórmulas de asimetría

def procesar_maximos (cuad_der,cuad_izq,isq_der,isq_izq):
    relacionFlex=(isq_der/isq_izq)*100-100
    simetricoFlex=0
    asimetricoDFlex=0
    asimetricoIFlex=0
    simetricoFlex=(relacionFlex <=10) & (relacionFlex >=-10)
    asimetricoDFlex=(relacionFlex <-10) & (relacionFlex >=-25)
    asimetricoIFlex=(relacionFlex >10) & (relacionFlex <25)
    #relacion extensores
    simetricoExt=0
    asimetricoDExt=0
    asimetricoIExt=0

    relacionExt=(cuad_der/cuad_izq)*100-100

    simetricoExt=(relacionExt <=10) & (relacionExt >=-10)
    asimetricoDExt=(relacionExt <-10) & (relacionExt >=-25)
    asimetricoIExt=(relacionExt >10) & (relacionExt <25)

    #relacion interpierna derecha
    criticoD=(cuad_der+isq_der)*0.06
    sCriticoD=(cuad_der+isq_der)*0.3
    relacionMd=(cuad_der+isq_der)*0.7-cuad_der

    simetricoMd=0
    asimetricoDIsq=0
    asimetricoDCuad=0
    sAsimetricoDIsq=0
    sAsimetricoDCuad=0

    simetricoMd=(relacionMd <=criticoD) & (relacionMd >=criticoD*-1)
    asimetricoDIsq=(relacionMd <criticoD*-1) & (relacionMd >=sCriticoD*-1)
    asimetricoDCuad=(relacionMd >criticoD) & (relacionMd <sCriticoD)
    sAsimetricoDIsq=(relacionMd <=sCriticoD*-1)
    sAsimetricoDCuad=(relacionMd >= sCriticoD)

    #relacion interpierna Izquierda
    criticoI=(cuad_izq+isq_izq)*0.06
    sCriticoI=(cuad_izq+isq_izq)*0.3
    relacionMi=(cuad_izq+isq_izq)*0.7-cuad_izq

    simetricoMi=0
    asimetricoIIsq=0
    asimetricoICuad=0
    sAsimetricoIIsq=0
    sAsimetricoICuad=0

    simetricoMi=(relacionMi <=criticoI) & (relacionMi >=criticoI*-1)
    asimetricoIIsq=(relacionMi <criticoI*-1) & (relacionMi >=sCriticoI*-1)
    asimetricoICuad=(relacionMi >criticoI) & (relacionMi <sCriticoI)
    sAsimetricoIIsq=(relacionMi <=sCriticoI*-1)
    sAsimetricoICuad=(relacionMi >= sCriticoI)

    print("2) FUERZA DE MUSLOS\n 2.1) RELACION FLEXORES (-10(ok)<10-25(leve)<+25(grave))")
    if simetricoFlex !=0:
      print('Valores Óptimos en flexores \n Porcentaje de asiemtría =',relacionFlex)
    elif asimetricoDFlex !=0:
      print('Fortalecer flexores D \n Porcentaje de asiemtría =',relacionFlex)
    else:
      print('Fortalecer flexores I \n Porcentaje de asiemtría =',relacionFlex)


    print("2.2) RELACION EXTENSORES (-10(ok)<10-25(leve)<+25(grave))")
    if simetricoExt !=0:
      print('Valores Óptimos en Extensores \n Porcentaje de asiemtría =',relacionExt)
    elif asimetricoDExt !=0:
      print('Fortalecer Extensores D \n Porcentaje de asiemtría =',relacionExt)
    else:
      print('Fortalecer Extensores I \n Porcentaje de asiemtría =',relacionExt)


    print("2.3) RELACION MUSLO DERECHO (-",criticoD,"(ok)<",criticoD,"-",sCriticoD,"(leve)<+",sCriticoD,"(grave))")
    if simetricoMd !=0:
      print('Valores Óptimos en Muslo derecho \n Porcentaje de relación =',relacionMd)
    elif asimetricoDIsq !=0 or sAsimetricoDIsq!=0:
      print('Fortalecer Flexores D \n Porcentaje de relación =',relacionMd)
    else:
      print('Fortalecer Extensores D \n Porcentaje de relación =',relacionMd)


    print("2.4) RELACION MUSLO Izquierdo (-",criticoI,"(ok)<",criticoI,"-",sCriticoI,"(leve)<+",sCriticoI,"(grave))")
    if simetricoMi !=0:
      print('Valores Óptimos en Muslo Izquierdo \n Porcentaje de relación =',relacionMi)
    elif asimetricoIIsq !=0 or sAsimetricoIIsq!=0:
      print('Fortalecer Flexores I \n Porcentaje de relación =',relacionMi)
    else:
      print('Fortalecer Extensores I \n Porcentaje de relación =',relacionMi)
    return
i=0



Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
Cuadriceps Derechos
Isquios Derechos
